In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

con = sqlite3.connect("database.sqlite")

# Load the data into a DataFrame
df_country = pd.read_sql_query("SELECT * from country", con)
df_league = pd.read_sql_query("SELECT * from league", con)
df_match = pd.read_sql_query("SELECT * from match", con)
df_player = pd.read_sql_query("SELECT * from player", con)
df_player_attributes = pd.read_sql_query("SELECT * from player_attributes", con)
df_team = pd.read_sql_query("SELECT * from team", con)
df_team_attributes = pd.read_sql_query("SELECT * from team_attributes", con)

con.close()